In [5]:
import ngmix
import numpy as np
import joblib


In [15]:
def gen_data(n, seed, sigma_e=0.2, g_true=0.02, R=0.5):
    rng = np.random.RandomState(seed=seed)
    e = rng.normal(size=n, scale=sigma_e)
    g = e + R*g_true
    return g, R

def weight_fun(g):
    prior = ngmix.priors.GPriorBA(0.3, rng=np.random.RandomState())
    pvals = prior.get_prob_array2d(g, 0)
    return pvals

def meas_data_mn(g, R):
    w = weight_fun(g)
    
    g_p = g + R*0.01
    w_p = weight_fun(g_p)
    
    g_m = g - R*0.01
    w_m = weight_fun(g_m)
    
    return np.sum(w*g), np.sum(w), np.sum(w_p * g_p), np.sum(w_p), np.sum(w_m * g_m), np.sum(w_m)

In [16]:
def _fun(n, seed):
    g, R = gen_data(n, seed)
    return meas_data_mn(g, R)

In [31]:
n_tot = 1_000_000_000
n_jobs = 200
n_per = n_tot // n_jobs

seeds = np.random.RandomState(seed=132).randint(1, 2**30, size=n_jobs)

jobs = [
    joblib.delayed(_fun)(n_per, seed)
    for seed in seeds
]

with joblib.Parallel(n_jobs=-1, verbose=100) as par:
    all_sums = par(jobs)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   21.4s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   21.7s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:   21.7s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   21.9s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   22.0s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:   22.0s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:   22.1s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:   22.3s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   22.5s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   22.6s
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:   22.6s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:   22.7s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:   22.8s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:   22.9s
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed:  

[Parallel(n_jobs=-1)]: Done 133 tasks      | elapsed:   52.7s
[Parallel(n_jobs=-1)]: Done 134 tasks      | elapsed:   52.8s
[Parallel(n_jobs=-1)]: Done 135 tasks      | elapsed:   52.9s
[Parallel(n_jobs=-1)]: Done 136 tasks      | elapsed:   53.3s
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed:   54.8s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:   54.8s
[Parallel(n_jobs=-1)]: Done 139 tasks      | elapsed:   54.9s
[Parallel(n_jobs=-1)]: Done 140 tasks      | elapsed:   55.2s
[Parallel(n_jobs=-1)]: Done 141 tasks      | elapsed:   55.3s
[Parallel(n_jobs=-1)]: Done 142 tasks      | elapsed:   55.5s
[Parallel(n_jobs=-1)]: Done 143 tasks      | elapsed:   55.6s
[Parallel(n_jobs=-1)]: Done 144 tasks      | elapsed:   55.7s
[Parallel(n_jobs=-1)]: Done 145 tasks      | elapsed:   55.9s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   56.3s
[Parallel(n_jobs=-1)]: Done 147 tasks      | elapsed:   56.3s
[Parallel(n_jobs=-1)]: Done 148 tasks      | elapsed:   56.4s
[Paralle

In [32]:
def _jk_est(all_sums):
    all_sums = np.array(all_sums)
    n = all_sums.shape[0]
    gjk = np.zeros(n)
    inds = np.arange(n)
    for i in range(n):
        msk = inds != i
        sums = np.sum(all_sums[msk, :], axis=0)
        g1 = sums[0]/sums[1]
        g1_1p = sums[2]/sums[3]
        g1_1m = sums[4]/sums[5]
        R11 = (g1_1p - g1_1m)/0.02

        gjk[i] = g1/R11
        
    mn = np.mean(gjk)
    dgjk = gjk - mn
    sd = np.sqrt((n-1)/n*np.sum(dgjk**2))
    return mn, sd

In [33]:
g_mn, g_sd = _jk_est(all_sums)

In [34]:
m_mn, m_sd = g_mn/0.02-1, g_sd/0.02

In [35]:
print("m = %0.3f +/- %0.3f [10^-3, 3sigma]" % (
    m_mn/1e-3,
    m_sd*3/1e-3
))

m = -0.156 +/- 2.213 [10^-3, 3sigma]
